# Import Statements

In [ ]:
pip install opencage geopy requests

In [ ]:
import pandas as pd
import re
from opencage.geocoder import OpenCageGeocode
import time
import random
import numpy as np
import plotly.express as px

# Load the Data

In [ ]:
df_data = pd.read_csv('riga_listings.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`?
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
df_data.shape

(1327, 7)

In [ ]:
df_data.columns

Index(['Title', 'Location', 'Rooms', 'Size', 'Floor', 'Series', 'Price'], dtype='object')

In [ ]:
df_data.isna().any()

Title       False
Location    False
Rooms       False
Size        False
Floor       False
Series      False
Price       False
dtype: bool

In [ ]:
df_data.duplicated().value_counts()

False    1322
True        5
Name: count, dtype: int64

In [ ]:
df_data[df_data.duplicated() == True]

,Title,Location,Rooms,Size,Floor,Series,Price
234,Izīrē ilgtermiņā 2 istabu dzīvokli pilsētas ce...,centrsMaiznīcas 8,2,50,1/2,P. kara,380 €/mēn.
494,"Сдам квартиру, желательно на длительный срок. ...",MežciemsEizenšteina 19,2,49,4/5,LT proj.,400 €/mēn.
804,Аренда на короткий срок. Посуточная аренда нов...,PļavniekiDzeņu 1,2,50,3/9,602.,45 €/dienā
1070,"Сдаем 1-комнатную квартиру, со всеми удобствам...",ImantaJūrmalas g. 115b,1,50,2/2,Priv. m.,275 €/mēn.
1172,Atsevišķa ieeja - arī ideāli saistītam ar medi...,ĀgenskalnsLiepājas 40k12,3,70,1/4,Specpr.,195 €/mēn.


# Data Cleaning
* Remove Duplicate Listings
* Clean Location Data
* Clean Price Data
* Create latitude and longitude columns for visualization purposes

## Remove Duplicate Listings

In [ ]:
df_data[df_data['Location'] == 'centrsMaiznīcas 8']

,Title,Location,Rooms,Size,Floor,Series,Price
224,Izīrē ilgtermiņā 2 istabu dzīvokli pilsētas ce...,centrsMaiznīcas 8,2,50,1/2,P. kara,380 €/mēn.
234,Izīrē ilgtermiņā 2 istabu dzīvokli pilsētas ce...,centrsMaiznīcas 8,2,50,1/2,P. kara,380 €/mēn.


In [ ]:
df_data = df_data.drop_duplicates()

In [ ]:
df_data.duplicated().value_counts()

False    1322
Name: count, dtype: int64

## Clean Location Data
* Create a function to split the location column into address and location columns
* Change address column to contain full address including city and country

In [ ]:
df_data.sample(30)
df_data[df_data['Location'] == 'PurvciemsVaidavas 6 k-2/3']

,Title,Location,Rooms,Size,Floor,Series,Price
722,Сдаётся уютная 2х комнатная квартира в Пурвцие...,PurvciemsVaidavas 6 k-2/3,2,53,1/5,Jaun.,250 €/mēn.


Here we have a function to take in row location data and split the string into a district and address column according to the row index. Finding the correct regex through a lot of trial and error since the data has a lot variance in location string structure.

Examples:

* centrs	Bruņinieku 115B
* centrsPulkv.Brieža 10
* ImantaKurzemes pr. 120
* Šampēteris-PleskodāleZalves 43

So I needed a regex to find the last capitilized letter which is preceeded by a lower-case letter not a number, special character [".", "-" etc..]

Once case where the regex doesnt work is:

* centrs13. janvāra 21

Since the name of the street starts with a number which is an outlier in the dataset.

In [ ]:
def split_at_last_capitalized_letter(s):
    # Regex to find the last capitalized letter (preceded by a non-capitalized letter) followed by at least 1 non-capitalized letter and any combination of letters, numbers, and characters
    match = re.search(r'(.+[a-zāčēģīķļņšūž])([A-ZĀČĒĢĪĶĻŅŠŪŽ][\w\W]*)$', s)
    if match:
        district = match.group(1)
        address = match.group(2)
        return district, address
    else:
        # If no match is found, return the original string and an empty string
        return s, 'N/A'

for index, row in df_data.iterrows():
  location_series = split_at_last_capitalized_letter(row['Location'])
  df_data.loc[index, 'District'], df_data.loc[index, 'Address'] = location_series

In [ ]:
df_data.loc[df_data['District'] == 'centrs13. janvāra 21', 'Address'] = '13. janvāra 21'
df_data.loc[df_data['Address'] == '13. janvāra 21', 'District'] = 'centrs'

In [ ]:
df_data['District'].value_counts()

District
centrs                    616
Āgenskalns                 79
Purvciems                  67
Teika                      53
Imanta                     47
Pļavnieki                  40
Ķengarags                  38
Sarkandaugava              36
Vecrīga                    35
Maskavas priekšpilsēta     34
Ziepniekkalns              33
Šampēteris-Pleskodāle      31
Krasta r-ns                23
Iļģuciems                  21
Jugla                      19
Mežciems                   17
Zolitūde                   15
Mežaparks                  14
Vecmīlgrāvis               14
Dārzciems                  13
Čiekurkalns                12
Grīziņkalns                10
Klīversala                  8
Torņakalns                  8
Dzegužkalns                 7
Mangaļi                     7
Ķīpsala                     5
Bolderāja                   4
Daugavgrīva                 4
Katlakalns                  2
Šķirotava                   2
Mangaļsala                  2
Stacija-Tirgus              1
B

In [ ]:
df_data.drop(columns='Location', inplace=True)

In [ ]:
df_data['FullAddress'] = df_data['Address'] + ', Riga, Latvia'
df_data.drop(columns='Address', inplace=True)

## Clean Price Data
* Some listings are made as a daily price, we will convert it to monthly.
* Convert price column to integer dtype.

In [ ]:
def convert_price(price_str):

  price_str = price_str.replace(',', '')

  if 'dienā' in price_str:
    price_per_day = re.findall(r'\d+', price_str)[0]
    monthly_price = int(price_per_day) * 30
  else:
    monthly_price = re.findall(r'\d+', price_str)[0]
  return int(monthly_price)

df_data['Price'] = df_data['Price'].apply(convert_price)

In [ ]:
df_data['Price'].describe()

count     1322.000000
mean       683.401664
std        742.377976
min         30.000000
25%        300.000000
50%        460.000000
75%        850.000000
max      12900.000000
Name: Price, dtype: float64

We can see prices listed for 30/month and 12900/month, these have been errors made by the person listing the post. We will have to find listing erros and get rid of these anomalies.
* Daily price listed as monthly
* Monthly price listed as daily

In [ ]:
df_data.loc[df_data['Price'] == 30, 'Price'] = 900
df_data.loc[df_data['Price'] == 12900, 'Price'] = 12900 / 30
df_data.loc[df_data['Price'] == 10500, 'Price'] = 10500 / 30

## Create latitude and longitude columns for visualization purposes

In [ ]:
df_data['Latitude'] = np.nan
df_data['Longitude'] = np.nan

df_data[df_data['Latitude'].isna()]

,Title,Rooms,Size,Floor,Series,Price,District,FullAddress,Latitude,Longitude
0,"Bez starpniekiem un bez komisijas maksas, tieš...",3,70,7/9,Jaun.,830,centrs,"Klijānu 6, Riga, Latvia",NaN,NaN
1,Izīrē nelielu dzīvokli jaunbūvē Rīgas centrā -...,1,26,2/5,Jaun.,580,centrs,"Stabu 2a, Riga, Latvia",NaN,NaN
2,"Izīrē nelielu, bet pilnībā aprīkotu dzīvokli p...",1,20,2/5,Renov.,400,centrs,"Lāčplēša 53, Riga, Latvia",NaN,NaN
3,Izīrē mājīgu 75 kvadrātmetru divstāvīgu dzīvok...,3,75,2/2,P. kara,490,centrs,"Hospitāļu 5a, Riga, Latvia",NaN,NaN
4,Tiek izīrēts ļoti saulains un silts trīs istab...,3,63,5/9,602.,450,Mežciems,"Eizenšteina 31, Riga, Latvia",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1322,"Istabas 16M²(Caurstaigājama), 11M², 2, 5m²apģē...",2,40,1/5,Hrušč.,350,Mangaļi,"Ostas pr. 6, Riga, Latvia",NaN,NaN
1323,Uz īri tiek piedāvāts 2-istabu dzīvoklis unikā...,2,69,3/6,Jaun.,1400,Vecrīga,"Vāgnera 11, Riga, Latvia",NaN,NaN
1324,Uz īri tiek piedāvāts 3-istabu dzīvoklis unikā...,3,123,4/6,Jaun.,3500,Vecrīga,"Vāgnera 11, Riga, Latvia",NaN,NaN
1325,Izīrē 1 istabu 4-istabu dzīvoklī. Slēdzama kāp...,1,15,5/5,P. kara,175,centrs,"Briāna 20, Riga, Latvia",NaN,NaN


In [ ]:
api_key = "my_api_key_would_be_here :)"
geolocator = OpenCageGeocode(api_key)

def get_lat_lon(address):
  try:
    location = geolocator.geocode(address)
    if location:
      print(location[0]['geometry']['lat'], location[0]['geometry']['lng'])
      return location[0]['geometry']['lat'], location[0]['geometry']['lng']
    else:
      return None, None
  except Exception as e:
    print(f"Error: {e}")
    return None, None

def retry_missing_coords(df):
    current_listing = 0
    for index, row in df.iterrows():
        retry_count = 0
        max_retries = 5
        current_listing += 1
        while (pd.isna(row['Latitude']) or pd.isna(row['Longitude'])) and retry_count < max_retries:
            lat, lon = get_lat_lon(row['FullAddress'])
            print(f"Retrying: {row['FullAddress']}, Attempt: {retry_count + 1}")
            if lat is not None and lon is not None:
                print(f"Successfull. Now on: {current_listing}")
                df.at[index, 'Latitude'] = lat
                df.at[index, 'Longitude'] = lon
                break
            else:
                retry_count += 1
                # Exponential backoff with jitter
                wait_time = (2 ** retry_count) + random.uniform(0, 1)
                print(f"Waiting for {wait_time} seconds before next retry.")
                time.sleep(wait_time)
        if retry_count == max_retries:
            print(f"Failed to get coordinates for {row['FullAddress']} after {max_retries} attempts.")

retry_missing_coords(df_data)


In [ ]:
df_data[df_data['Longitude'].isna()]
df_data['District'][(df_data['Latitude'] == 56.946000) & (df_data['Longitude'] == 24.105890)].value_counts()

Series([], Name: count, dtype: int64)

Looks like geocoding had trouble with 94 addresses, which we will try to assign a district coordinate instead.

In [ ]:
district_geolocations = {
    'centrs': (56.9496, 24.1052),
    'Imanta': (56.9519, 24.0148),
    'Čiekurkalns': (56.9771, 24.1811),
    'Āgenskalns': (56.9386, 24.0752),
    'Mežaparks': (56.9818, 24.1395),
    'Purvciems': (56.9498, 24.1576),
    'Ķīpsala': (56.9496, 24.0898),
    'Sarkandaugava': (56.9835, 24.1353),
    'Mangaļi': (57.0016, 24.1445),
    'Jugla': (56.9764, 24.2099),
    'Pļavnieki': (56.9348, 24.1935),
    'Torņakalns': (56.9364, 24.0853),
    'Dzegužkalns': (56.9451, 24.0841),
    'Kundziņsala': (56.9985, 24.1061),
    'Iļģuciems': (56.9423, 24.0517),
    'Vecrīga': (56.9489, 24.1064),
    'Teika': (56.9753, 24.1917),
    'Klīversala': (56.9402, 24.0931),
    'Vecmīlgrāvis': (57.0314, 24.1365),
    'Zasulauks': (56.9401, 24.0728),
    'Šampēteris-Pleskodāle': (56.9293, 24.0533),
}

# Function to update latitude and longitude based on district
def update_geolocation(row):
    if (row['Latitude'] == 56.946) and (row['Longitude'] == 24.10589):
        district = row['District']
        if district in district_geolocations:
            return district_geolocations[district]
    return row['Latitude'], row['Longitude']

# Apply the function to update the DataFrame
df_data[['Latitude', 'Longitude']] = df_data.apply(update_geolocation, axis=1, result_type='expand')

df_data

,Title,Rooms,Size,Floor,Series,Price,District,FullAddress,Latitude,Longitude
0,"Bez starpniekiem un bez komisijas maksas, tieš...",3,70,7/9,Jaun.,830,centrs,"Klijānu 6, Riga, Latvia",56.972656,24.149312
1,Izīrē nelielu dzīvokli jaunbūvē Rīgas centrā -...,1,26,2/5,Jaun.,580,centrs,"Stabu 2a, Riga, Latvia",56.959529,24.121525
2,"Izīrē nelielu, bet pilnībā aprīkotu dzīvokli p...",1,20,2/5,Renov.,400,centrs,"Lāčplēša 53, Riga, Latvia",56.952321,24.130213
3,Izīrē mājīgu 75 kvadrātmetru divstāvīgu dzīvok...,3,75,2/2,P. kara,490,centrs,"Hospitāļu 5a, Riga, Latvia",56.969687,24.135034
4,Tiek izīrēts ļoti saulains un silts trīs istab...,3,63,5/9,602.,450,Mežciems,"Eizenšteina 31, Riga, Latvia",56.968072,24.232766
...,...,...,...,...,...,...,...,...,...,...
1322,"Istabas 16M²(Caurstaigājama), 11M², 2, 5m²apģē...",2,40,1/5,Hrušč.,350,Mangaļi,"Ostas pr. 6, Riga, Latvia",57.001600,24.144500
1323,Uz īri tiek piedāvāts 2-istabu dzīvoklis unikā...,2,69,3/6,Jaun.,1400,Vecrīga,"Vāgnera 11, Riga, Latvia",56.948488,24.111243
1324,Uz īri tiek piedāvāts 3-istabu dzīvoklis unikā...,3,123,4/6,Jaun.,3500,Vecrīga,"Vāgnera 11, Riga, Latvia",56.948488,24.111243
1325,Izīrē 1 istabu 4-istabu dzīvoklī. Slēdzama kāp...,1,15,5/5,P. kara,175,centrs,"Briāna 20, Riga, Latvia",56.964585,24.131486


In [ ]:
# Create a scatter plot on a Mapbox map
fig = px.scatter_mapbox(
    df_data,
    lat="Latitude",
    lon="Longitude",
    hover_name="Title",
    hover_data={"District": True, "Price": True, "Rooms": True, "Size": True, "FullAddress": True},
    color="Price",
    color_continuous_scale='Bluered',
    size="Price",
    zoom=9,
    height=600,
)

# Update the layout to use an appropriate Mapbox style
fig.update_layout(
    mapbox_style="open-street-map",
    title="Real Estate Listings in Riga"
)

# Show the plot
fig.show()

# Save data to .pkl format to keep column data types intact as opposed to csv/xlsx file types.

In [ ]:
df_data.to_pickle('riga_listings_data.pkl')

Size
50     64
30     55
40     39
42     36
25     36
       ..
224     1
105     1
117     1
144     1
263     1
Name: count, Length: 153, dtype: int64